In [1]:
import glob 
import csv
import pandas as pd
import numpy as np
from scipy import stats 
from sklearn.linear_model import LinearRegression
from sklearn import datasets
import plotly.graph_objects as go
import plotly.express as px
import statsmodels.api as sm
from scipy import stats
import seaborn as sns

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

In [2]:
merged_csvs = glob.glob('../raw_data_files_clean/merged_*')
merged_csvs.sort()

merged_csvs

['../raw_data_files_clean/merged_act_school.csv',
 '../raw_data_files_clean/merged_assessment.csv',
 '../raw_data_files_clean/merged_census_school.csv',
 '../raw_data_files_clean/merged_chronic_absenteeism_school.csv',
 '../raw_data_files_clean/merged_profile.csv',
 '../raw_data_files_clean/merged_tvaas_composite_school.csv',
 '../raw_data_files_clean/merged_tvaas_composite_subject_school.csv']

In [3]:
def pull_files_and_create_df(raw_file_names): 
    df_list=[]
    for i in raw_file_names:
        name = i.split('/')[2].split('.')[0]
        globals()[f'df_{name}'] = pd.read_csv(i, low_memory=False)
        d = df_list.append(f'df_{name}')
    df_list.sort(reverse=True)
    return df_list

In [4]:
pull_files_and_create_df(merged_csvs)

['df_merged_tvaas_composite_subject_school',
 'df_merged_tvaas_composite_school',
 'df_merged_profile',
 'df_merged_chronic_absenteeism_school',
 'df_merged_census_school',
 'df_merged_assessment',
 'df_merged_act_school']

In [5]:
df_merged_assessment['subject_group']='Other'
df_merged_assessment.loc[df_merged_assessment['subject']=='Algebra I', 'subject_group'] = 'Math'
df_merged_assessment.loc[df_merged_assessment['subject']=='Algebra II', 'subject_group'] = 'Math'
df_merged_assessment.loc[df_merged_assessment['subject']=='Integrated Math I', 'subject_group'] = 'Math'
df_merged_assessment.loc[df_merged_assessment['subject']=='Integrated Math II', 'subject_group'] = 'Math'
df_merged_assessment.loc[df_merged_assessment['subject']=='Integrated Math III', 'subject_group'] = 'Math'
df_merged_assessment.loc[df_merged_assessment['subject']=='Geometry', 'subject_group'] = 'Math'
df_merged_assessment.loc[df_merged_assessment['subject']=='Math', 'subject_group'] = 'Math'

df_merged_assessment.loc[df_merged_assessment['subject']=='English I', 'subject_group'] = 'ELA'
df_merged_assessment.loc[df_merged_assessment['subject']=='English II', 'subject_group'] = 'ELA'
df_merged_assessment.loc[df_merged_assessment['subject']=='English III', 'subject_group'] = 'ELA'
df_merged_assessment.loc[df_merged_assessment['subject']=='ELA', 'subject_group'] = 'ELA'

In [6]:
df_chronic_absenteeism = df_merged_chronic_absenteeism_school.loc[df_merged_chronic_absenteeism_school['student_group']=='All Students']

In [7]:
df_chronic_absenteeism

,grade_band,n_chronically_absent,n_students,pct_chronically_absent,school_number,school_name,student_group,district_number,district_name,file_source,file_year,updated_district_number,school_type,status
0,All Grades,245,1062,23.1,2,Anderson County High School,All Students,10,Anderson County,df_chronic_absenteeism_school_2022,2022,10,Public,A
11,All Grades,**,34,**,4,Anderson County Innovation Academy,All Students,10,Anderson County,df_chronic_absenteeism_school_2022,2022,10,Public Virtual School,A
18,All Grades,41,291,14.1,5,Andersonville Elementary,All Students,10,Anderson County,df_chronic_absenteeism_school_2022,2022,10,Public,A
28,All Grades,23,99,23.2,10,Briceville Elementary,All Students,10,Anderson County,df_chronic_absenteeism_school_2022,2022,10,Public,A
34,All Grades,95,444,21.4,15,Claxton Elementary,All Students,10,Anderson County,df_chronic_absenteeism_school_2022,2022,10,Public,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103571,All Grades,90,271,33.2,8120,Memphis Scholars Florida Kansas,All Students,985,Achievement School District,df_chronic_absenteeism_school_2017,2017,985,Public Charter,A
103576,All Grades,113,409,27.6,8125,Memphis Scholars Caldwell-Guthrie,All Students,985,Achievement School District,df_chronic_absenteeism_school_2017,2017,985,Public Charter,A
103582,All Grades,79,193,40.9,8130,Memphis Scholars Raleigh-Egypt,All Students,985,Achievement School District,df_chronic_absenteeism_school_2017,2017,985,Public Charter,A
103590,All Grades,61,412,14.8,8135,Kirby Middle School,All Students,985,Achievement School District,df_chronic_absenteeism_school_2017,2017,985,Public Charter,A


In [8]:
df_merged_profile['prev_year'] = df_merged_profile['file_year']
df_merged_profile['file_year'] = df_merged_profile['file_year']+1
df_merged_profile

,african_american_pct,asian_pct,district_number,district_name,economically_disadvantaged_pct,female_pct,hawaiian_pacisld_pct,hispanic_pct,limited_english_proficient_pct,male_pct,native_american_pct,school_number,school_name,students_with_disabilities_pct,total,white_pct,file_source,file_year,updated_district_number,school_type,status,prev_year
0,Less than 5%,Less than 5%,10,Anderson County,23,49.1,Less than 5%,Less than 5%,Less than 5%,50.9,Less than 5%,2,Anderson County High School,14.1,1095,Greater than 95%,df_profile_school_2021,2022,10.0,Public,A,2021
1,Less than 5%,Less than 5%,10,Anderson County,35.6,53.8,*,Less than 5%,Less than 5%,46.2,Less than 5%,5,Andersonville Elementary,16.8,292,93.8,df_profile_school_2021,2022,10.0,Public,A,2021
2,*,*,10,Anderson County,55.1,46.7,*,Less than 5%,*,53.3,Less than 5%,10,Briceville Elementary,28,107,Greater than 95%,df_profile_school_2021,2022,10.0,Public,A,2021
3,Less than 5%,Less than 5%,10,Anderson County,45,47.3,Less than 5%,7.9,5.9,52.7,Less than 5%,15,Claxton Elementary,19.3,404,86.4,df_profile_school_2021,2022,10.0,Public,A,2021
4,Less than 5%,Less than 5%,10,Anderson County,34.7,52.3,Less than 5%,Less than 5%,Less than 5%,47.7,Less than 5%,20,Clinton Middle School,14.6,591,89,df_profile_school_2021,2022,10.0,Public,A,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22078,24.0,2.3,0,State of Tennessee,36.1,48.7,0.2,10.4,4.6,51.3,0.4,0,All Schools,13.6,975222,62.7,df_profile_district_2018,2019,NaN,NaN,NaN,2018
22079,24.0970102370386,1.89598653317976,0,State of Tennessee,58.6170113240702,48.6308665641084,NaN,7.28543295988144,4.33614294804159,51.3686300409965,0.295392124487544,0,All Schools,13.6845888673212,993256.0,66.2698236909719,df_profile_district_2013,2014,NaN,NaN,NaN,2013
22080,0.23601865169344477,0.0179484160904126,0,State of Tennessee,0.5835451525559064,0.48547635012361096,NaN,0.06642336887255529,0.00030966724846573953,0.514523649876389,0.0018784773584615937,0,All Schools,0.14514334266600468,976855.0,0.6777310859851258,df_profile_district_2012,2013,NaN,NaN,NaN,2012
22081,0.2420811782537569,0.019042839125125346,0,State of Tennessee,0.5640112479876741,0.48574016026479055,NaN,0.062310726160387006,0.0003305874539055902,0.5142598397352095,0.0027729274921535567,0,All Schools,0.13977237551128355,998223.0,0.6737923289685772,df_profile_district_2011,2012,NaN,NaN,NaN,2011


In [9]:
df = df_merged_assessment.loc[(df_merged_assessment['student_group']=='All Students') 
                         & (df_merged_assessment['grade']=='All Grades')
                         & (df_merged_assessment['test']!='MSAA/Alt-Science/Social Studies')
                         & (df_merged_assessment['test']!='MSAA')
                         & (df_merged_assessment['school_type'].notnull())
                        ]

In [11]:
df = df.replace(['*','**','Less than 1%','Less than 5%','Greater than 95%'],[0,0,0,0,'95'])
df['valid_tests'] = df['valid_tests'].astype('float64') 
df['pct_met_exceeded'] = df['pct_met_exceeded'].astype('float64') 
df['num_met_or_exceeded'] = df['valid_tests'] * df['pct_met_exceeded']

In [12]:
df = (
    df[['district_number', 'district_name', 'school_number', 'subject_group',
         'school_name','file_year', 'updated_district_number', 'school_type',
         'valid_tests','num_met_or_exceeded'
        ]]
    .groupby(by=['district_number', 'district_name', 'school_number', 'subject_group',
         'school_name','file_year', 'updated_district_number', 'school_type'])
    .sum(['num_met_or_exceeded','valid_tests'])
    .reset_index()
)
df['pct_met_exceeded'] = df['num_met_or_exceeded']/df['valid_tests']

In [13]:
df = df.merge(
        df_merged_profile, 
        how='left', 
        left_on=['file_year','district_number','school_number'], 
        right_on=['file_year','district_number','school_number'])

In [14]:
df = df.merge(
    df_chronic_absenteeism,
    how='left',
        left_on=['file_year','district_number','school_number'], 
        right_on=['file_year','district_number','school_number'])

In [15]:
df_econ_v_percent = (
    df
    .loc[(df['updated_district_number']==190) 
         & (df['file_year']==2022)]
    [['school_number','district_number','school_type','pct_met_exceeded','economically_disadvantaged_pct','subject_group']]
)

In [16]:
df_poc_v_percent = (
    df
    .loc[(df['updated_district_number']==190) 
         & (df['file_year']==2022)]
    [['school_number','district_number','school_type','pct_met_exceeded','hispanic_pct','african_american_pct','native_american_pct','subject_group']]
)

In [17]:
df.columns

Index(['district_number', 'district_name_x', 'school_number', 'subject_group',
       'school_name_x', 'file_year', 'updated_district_number_x',
       'school_type_x', 'valid_tests', 'num_met_or_exceeded',
       'pct_met_exceeded', 'african_american_pct', 'asian_pct',
       'district_name_y', 'economically_disadvantaged_pct', 'female_pct',
       'hawaiian_pacisld_pct', 'hispanic_pct',
       'limited_english_proficient_pct', 'male_pct', 'native_american_pct',
       'school_name_y', 'students_with_disabilities_pct', 'total', 'white_pct',
       'file_source_x', 'updated_district_number_y', 'school_type_y',
       'status_x', 'prev_year', 'grade_band', 'n_chronically_absent',
       'n_students', 'pct_chronically_absent', 'school_name', 'student_group',
       'district_name', 'file_source_y', 'updated_district_number',
       'school_type', 'status_y'],
      dtype='object')

In [18]:
df = df.replace(['*','**','Less than 1%','Less than 5%','Greater than 95%'],[0,0,0,0,'95'])
df = df.dropna()
cols = ['african_american_pct',
        'asian_pct',
        'economically_disadvantaged_pct',
        'hawaiian_pacisld_pct', 
        'hispanic_pct',
        'native_american_pct',
        'white_pct',
        'limited_english_proficient_pct', 
        'female_pct', 'male_pct', 
        'students_with_disabilities_pct'
       ]
col_names = ['african_american_num',
             'asian_num',
             'economically_disadvantaged_num',
             'hawaiian_pacisld_num', 
             'hispanic_num',
             'native_american_num',
             'white_num',
             'limited_english_proficient_num', 
             'female_num', 'male_num', 
             'students_with_disabilities_num']
for c, n in zip(cols,col_names):
    df[n] = df['total'].astype('float64') * df[c].astype('float64')/100 

In [19]:
reg_df = pd.get_dummies(data=df, columns=['school_type_x'])
reg_df = reg_df[['school_type_y',
                 'total',
    'subject_group',
    'school_number',
    'school_name',
    'updated_district_number',
    'african_american_num',
    'asian_num',
    'economically_disadvantaged_num',
    'hawaiian_pacisld_num', 
    'hispanic_num',
    'native_american_num',
    'white_num',
    'limited_english_proficient_num', 
    'female_num', 
    'male_num', 
    'students_with_disabilities_num',
    'n_chronically_absent',
    'school_type_x_Public',
    'school_type_x_Public Charter',
    'school_type_x_Public Virtual School',
    'school_type_x_State Special School',
    'file_year',
    'pct_met_exceeded']]
reg_df['pct_met_exceeded'] = reg_df['pct_met_exceeded'].astype('float64')
reg_df['n_chronically_absent'] = reg_df['n_chronically_absent'].astype('float64')
reg_df['total'] = reg_df['total'].astype('float64')
reg_df.corr()

,total,school_number,updated_district_number,african_american_num,asian_num,economically_disadvantaged_num,hawaiian_pacisld_num,hispanic_num,native_american_num,white_num,limited_english_proficient_num,female_num,male_num,students_with_disabilities_num,n_chronically_absent,school_type_x_Public,school_type_x_Public Charter,school_type_x_Public Virtual School,school_type_x_State Special School,file_year,pct_met_exceeded
total,1.000000,-0.118416,0.143284,0.429991,0.530823,0.484857,0.395944,0.515632,0.434848,0.769753,0.286359,0.995770,0.996047,0.850569,0.610045,0.161479,-0.129343,-0.078166,-0.062044,-0.149693,0.175406
school_number,-0.118416,1.000000,0.146199,0.325060,-0.082087,0.165559,-0.107259,0.070778,-0.144977,-0.353502,0.119699,-0.109949,-0.125854,-0.201641,0.091309,-0.837545,0.940647,-0.025374,-0.019549,0.043371,-0.306732
updated_district_number,0.143284,0.146199,1.000000,0.181826,0.148347,-0.014614,0.073747,-0.020108,0.061585,0.059303,-0.064622,0.145726,0.139751,0.086219,-0.001202,-0.108059,0.085165,0.034805,0.080312,-0.030914,0.098635
african_american_num,0.429991,0.325060,0.181826,1.000000,0.167661,0.673385,0.156147,0.359198,0.064561,-0.190605,0.273044,0.437906,0.418830,0.332066,0.475745,-0.135914,0.176596,-0.041624,-0.034865,-0.041304,-0.330235
asian_num,0.530823,-0.082087,0.148347,0.167661,1.000000,-0.021398,0.284777,0.273799,0.331921,0.379105,0.266549,0.534563,0.522937,0.314352,0.133488,0.090207,-0.084671,-0.023845,-0.018938,-0.143613,0.325478
economically_disadvantaged_num,0.484857,0.165559,-0.014614,0.673385,-0.021398,1.000000,0.106495,0.516426,0.109653,0.043068,0.410038,0.477159,0.488398,0.527517,0.631705,-0.001809,0.055498,-0.091625,-0.056062,-0.098199,-0.411357
hawaiian_pacisld_num,0.395944,-0.107259,0.073747,0.156147,0.284777,0.106495,1.000000,0.208927,0.528664,0.307327,0.073117,0.392205,0.396380,0.373976,0.105220,0.097138,-0.090176,-0.027605,-0.021038,-0.279035,0.138781
hispanic_num,0.515632,0.070778,-0.020108,0.359198,0.273799,0.516426,0.208927,1.000000,0.135212,0.092255,0.882846,0.505346,0.521411,0.398524,0.469550,-0.001185,0.030951,-0.049499,-0.033441,-0.019267,-0.147195
native_american_num,0.434848,-0.144977,0.061585,0.064561,0.331921,0.109653,0.528664,0.135212,1.000000,0.432618,0.016344,0.433012,0.433141,0.414292,0.105468,0.121641,-0.121469,-0.014042,-0.028455,-0.366720,0.182538
white_num,0.769753,-0.353502,0.059303,-0.190605,0.379105,0.043068,0.307327,0.092255,0.432618,1.000000,-0.102577,0.762549,0.770564,0.700593,0.308123,0.272082,-0.265971,-0.055051,-0.043661,-0.160038,0.423141


In [20]:
reg_df.loc[reg_df['school_name']=='KIPP Nashville Collegiate High School']

,school_type_y,total,subject_group,school_number,school_name,updated_district_number,african_american_num,asian_num,economically_disadvantaged_num,hawaiian_pacisld_num,hispanic_num,native_american_num,white_num,limited_english_proficient_num,female_num,male_num,students_with_disabilities_num,n_chronically_absent,school_type_x_Public,school_type_x_Public Charter,school_type_x_Public Virtual School,school_type_x_State Special School,file_year,pct_met_exceeded
5959,Public Charter,333.0,ELA,8011,KIPP Nashville Collegiate High School,190.0,225.108,0.999,157.842,0.0,93.906,0.0,12.987,15.984,172.161,160.839,37.962,56.0,0,1,0,0,2019,36.955285
5960,Public Charter,429.0,ELA,8011,KIPP Nashville Collegiate High School,190.0,274.131,3.861,181.896,0.0,139.854,0.0,11.154,62.205,215.787,213.213,63.063,153.0,0,1,0,0,2021,23.257959
5961,Public Charter,482.0,ELA,8011,KIPP Nashville Collegiate High School,190.0,307.998,0.000,211.116,0.0,150.866,0.0,0.000,55.912,241.000,241.000,75.192,126.0,0,1,0,0,2022,30.927966
5964,Public Charter,333.0,Math,8011,KIPP Nashville Collegiate High School,190.0,225.108,0.999,157.842,0.0,93.906,0.0,12.987,15.984,172.161,160.839,37.962,56.0,0,1,0,0,2019,24.385976
5965,Public Charter,429.0,Math,8011,KIPP Nashville Collegiate High School,190.0,274.131,3.861,181.896,0.0,139.854,0.0,11.154,62.205,215.787,213.213,63.063,153.0,0,1,0,0,2021,4.913295
5966,Public Charter,482.0,Math,8011,KIPP Nashville Collegiate High School,190.0,307.998,0.000,211.116,0.0,150.866,0.0,0.000,55.912,241.000,241.000,75.192,126.0,0,1,0,0,2022,7.613576
5968,Public Charter,333.0,Other,8011,KIPP Nashville Collegiate High School,190.0,225.108,0.999,157.842,0.0,93.906,0.0,12.987,15.984,172.161,160.839,37.962,56.0,0,1,0,0,2019,0.000000
5969,Public Charter,429.0,Other,8011,KIPP Nashville Collegiate High School,190.0,274.131,3.861,181.896,0.0,139.854,0.0,11.154,62.205,215.787,213.213,63.063,153.0,0,1,0,0,2021,13.507143
5970,Public Charter,482.0,Other,8011,KIPP Nashville Collegiate High School,190.0,307.998,0.000,211.116,0.0,150.866,0.0,0.000,55.912,241.000,241.000,75.192,126.0,0,1,0,0,2022,21.789091


In [21]:
math_reg_df = reg_df.loc[(reg_df['subject_group']=='Math') & (reg_df['updated_district_number']!=0)]
math_reg_df = math_reg_df.reset_index()
ela_reg_df = reg_df.loc[(reg_df['subject_group']=='ELA') & (reg_df['updated_district_number']!=0)]
ela_reg_df = ela_reg_df.reset_index()

In [22]:
# sns.pairplot(math_reg_df)

In [23]:
math_X = math_reg_df[['african_american_num',
    'asian_num',
    'economically_disadvantaged_num',
    'hawaiian_pacisld_num', 
    'hispanic_num',
    'native_american_num',
    # 'white_num',
    'limited_english_proficient_num', 
    #'female_num', 
    'students_with_disabilities_num',
    'n_chronically_absent',
    'school_type_x_Public Charter',
    'school_type_x_Public Virtual School',
    'school_type_x_State Special School',
                      'total'
    # 'file_year'
                            ]]
math_y = math_reg_df[['pct_met_exceeded']]

In [24]:
math_X.dtypes

african_american_num                   float64
asian_num                              float64
economically_disadvantaged_num         float64
hawaiian_pacisld_num                   float64
hispanic_num                           float64
native_american_num                    float64
limited_english_proficient_num         float64
students_with_disabilities_num         float64
n_chronically_absent                   float64
school_type_x_Public Charter             uint8
school_type_x_Public Virtual School      uint8
school_type_x_State Special School       uint8
total                                  float64
dtype: object

In [25]:
model = sm.OLS(math_y, math_X)
results_math_w_school_type = model.fit()
summary_math_w_school_type = results_math_w_school_type.summary()
print(summary_math_w_school_type)
predictions_math_with_school_type = results_math_w_school_type.predict()
results_as_html = summary_math_w_school_type.tables[1].as_html()
summary_math_w_school_type_df = pd.read_html(results_as_html, header=0, index_col=0)[0]

                                 OLS Regression Results                                
Dep. Variable:       pct_met_exceeded   R-squared (uncentered):                   0.681
Model:                            OLS   Adj. R-squared (uncentered):              0.681
Method:                 Least Squares   F-statistic:                              918.1
Date:                Tue, 30 Aug 2022   Prob (F-statistic):                        0.00
Time:                        19:55:34   Log-Likelihood:                         -24859.
No. Observations:                5593   AIC:                                  4.974e+04
Df Residuals:                    5580   BIC:                                  4.983e+04
Df Model:                          13                                                  
Covariance Type:            nonrobust                                                  
                                          coef    std err          t      P>|t|      [0.025      0.975]
----------------

In [26]:
math_X = math_reg_df[['african_american_num',
    'asian_num',
    'economically_disadvantaged_num',
    'hawaiian_pacisld_num', 
    'hispanic_num',
    'native_american_num',
    # 'white_num',
    'limited_english_proficient_num', 
    #'female_num', 
    'students_with_disabilities_num',
    'total',
    'n_chronically_absent',
    # 'school_type_x_Public Charter',
    # 'school_type_x_Public Virtual School',
    # 'school_type_x_State Special School'
    # 'file_year'
                            ]]
math_y = math_reg_df[['pct_met_exceeded']]

In [27]:
model = sm.OLS(math_y, math_X)
results_math_no_school_type = model.fit()
summary_math_no_school_type = results_math_no_school_type.summary()
print(summary_math_no_school_type)
predictions_math_no_school_type = results_math_no_school_type.predict()
results_as_html = summary_math_no_school_type.tables[1].as_html()
summary_math_no_school_type_df = pd.read_html(results_as_html, header=0, index_col=0)[0]

                                 OLS Regression Results                                
Dep. Variable:       pct_met_exceeded   R-squared (uncentered):                   0.676
Model:                            OLS   Adj. R-squared (uncentered):              0.676
Method:                 Least Squares   F-statistic:                              1166.
Date:                Tue, 30 Aug 2022   Prob (F-statistic):                        0.00
Time:                        19:55:34   Log-Likelihood:                         -24905.
No. Observations:                5593   AIC:                                  4.983e+04
Df Residuals:                    5583   BIC:                                  4.990e+04
Df Model:                          10                                                  
Covariance Type:            nonrobust                                                  
                                     coef    std err          t      P>|t|      [0.025      0.975]
---------------------

In [28]:
predictions_math_no_school_type_df = pd.DataFrame(predictions_math_no_school_type)
predictions_math_no_school_type_df = pd.merge(predictions_math_no_school_type_df, math_reg_df, left_index=True, right_index=True)
predictions_math_no_school_type_df

,0,index,school_type_y,total,subject_group,school_number,school_name,updated_district_number,african_american_num,asian_num,economically_disadvantaged_num,hawaiian_pacisld_num,hispanic_num,native_american_num,white_num,limited_english_proficient_num,female_num,male_num,students_with_disabilities_num,n_chronically_absent,school_type_x_Public,school_type_x_Public Charter,school_type_x_Public Virtual School,school_type_x_State Special School,file_year,pct_met_exceeded
0,50.602831,9,Public,1056.0,Math,2,Anderson County High School,10.0,9.504,6.336,251.328,0.000,11.616,3.168,1025.376,1.056,508.992,547.008,122.496,162.0,1,0,0,0,2019,32.515276
1,43.956031,10,Public,1078.0,Math,2,Anderson County High School,10.0,14.014,10.780,257.642,3.234,11.858,6.468,1031.646,2.156,526.064,551.936,154.154,235.0,1,0,0,0,2021,22.098126
2,48.684435,11,Public,1095.0,Math,2,Anderson County High School,10.0,0.000,0.000,251.850,0.000,0.000,0.000,1040.250,0.000,537.645,557.355,154.395,245.0,1,0,0,0,2022,23.043127
3,22.776684,26,Public,310.0,Math,5,Andersonville Elementary,10.0,5.890,0.000,113.150,0.000,3.100,3.100,297.910,0.930,141.050,168.950,62.930,26.0,1,0,0,0,2019,54.800000
4,20.869637,27,Public,296.0,Math,5,Andersonville Elementary,10.0,5.920,0.000,105.080,0.000,5.032,0.888,283.864,2.960,158.952,137.048,42.920,15.0,1,0,0,0,2021,57.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5588,-14.876304,27556,Public Charter,460.0,Math,8140,Hillcrest High School,985.0,452.180,0.000,231.840,0.000,7.820,0.000,0.000,15.180,224.940,235.060,82.800,288.0,0,1,0,0,2021,0.000000
5589,-5.790978,27557,Public Charter,451.0,Math,8140,Hillcrest High School,985.0,428.450,0.000,234.971,0.000,0.000,0.000,0.000,27.962,222.794,228.206,86.141,245.0,0,1,0,0,2022,0.000000
5590,-2.485562,27566,Public Charter,156.0,Math,8005,Bluff City High School,986.0,122.928,0.000,100.932,0.000,31.980,0.000,0.936,21.996,80.964,75.036,34.944,90.0,0,1,0,0,2019,3.494444
5591,-23.553529,27567,Public Charter,416.0,Math,8005,Bluff City High School,986.0,331.968,2.912,215.904,0.000,81.120,0.000,0.000,64.064,193.024,222.976,69.056,324.0,0,1,0,0,2021,0.000000


In [29]:
ela_X = ela_reg_df[['african_american_num',
    'asian_num',
    'economically_disadvantaged_num',
    'hawaiian_pacisld_num', 
    'hispanic_num',
    'native_american_num',
    # 'white_num',
    'limited_english_proficient_num', 
    #'female_num', 
    'students_with_disabilities_num',
    'total',
    'n_chronically_absent',
    'school_type_x_Public Charter',
    'school_type_x_Public Virtual School',
    'school_type_x_State Special School'
    # 'file_year'
                            ]]
ela_y = ela_reg_df[['pct_met_exceeded']]

In [30]:
model = sm.OLS(ela_y, ela_X)
results_ela_w_school_type = model.fit()
summary_ela_w_school_type = results_ela_w_school_type.summary()
print(summary_ela_w_school_type)
predictions_ela_with_school_type = results_ela_w_school_type.predict()
results_as_html = summary_ela_w_school_type.tables[1].as_html()
summary_ela_w_school_type_df = pd.read_html(results_as_html, header=0, index_col=0)[0]

                                 OLS Regression Results                                
Dep. Variable:       pct_met_exceeded   R-squared (uncentered):                   0.740
Model:                            OLS   Adj. R-squared (uncentered):              0.740
Method:                 Least Squares   F-statistic:                              1221.
Date:                Tue, 30 Aug 2022   Prob (F-statistic):                        0.00
Time:                        19:55:34   Log-Likelihood:                         -24116.
No. Observations:                5580   AIC:                                  4.826e+04
Df Residuals:                    5567   BIC:                                  4.834e+04
Df Model:                          13                                                  
Covariance Type:            nonrobust                                                  
                                          coef    std err          t      P>|t|      [0.025      0.975]
----------------

In [31]:
ela_X = ela_reg_df[['african_american_num',
    'asian_num',
    'economically_disadvantaged_num',
    'hawaiian_pacisld_num', 
    'hispanic_num',
    'native_american_num',
    # 'white_num',
    'limited_english_proficient_num', 
    #'female_num', 
    'students_with_disabilities_num',
    'total',
    'n_chronically_absent',
    # 'school_type_x_Public Charter',
    # 'school_type_x_Public Virtual School',
    # 'school_type_x_State Special School'
    # 'file_year'
                            ]]
ela_y = ela_reg_df[['pct_met_exceeded']]

In [32]:
model = sm.OLS(ela_y, ela_X)
results_ela_no_school_type = model.fit()
summary_ela_no_school_type = results_ela_no_school_type.summary()
print(summary_ela_no_school_type)
predictions_ela_no_school_type = results_ela_no_school_type.predict()
results_as_html = summary_ela_no_school_type.tables[1].as_html()
summary_ela_no_school_type_df = pd.read_html(results_as_html, header=0, index_col=0)[0]

                                 OLS Regression Results                                
Dep. Variable:       pct_met_exceeded   R-squared (uncentered):                   0.734
Model:                            OLS   Adj. R-squared (uncentered):              0.734
Method:                 Least Squares   F-statistic:                              1539.
Date:                Tue, 30 Aug 2022   Prob (F-statistic):                        0.00
Time:                        19:55:34   Log-Likelihood:                         -24180.
No. Observations:                5580   AIC:                                  4.838e+04
Df Residuals:                    5570   BIC:                                  4.845e+04
Df Model:                          10                                                  
Covariance Type:            nonrobust                                                  
                                     coef    std err          t      P>|t|      [0.025      0.975]
---------------------

In [33]:
predictions_ela_no_school_type_df = pd.DataFrame(predictions_ela_no_school_type)
predictions_ela_no_school_type_df = pd.merge(predictions_ela_no_school_type_df, ela_reg_df, left_index=True, right_index=True)
predictions_ela_no_school_type_df

,0,index,school_type_y,total,subject_group,school_number,school_name,updated_district_number,african_american_num,asian_num,economically_disadvantaged_num,hawaiian_pacisld_num,hispanic_num,native_american_num,white_num,limited_english_proficient_num,female_num,male_num,students_with_disabilities_num,n_chronically_absent,school_type_x_Public,school_type_x_Public Charter,school_type_x_Public Virtual School,school_type_x_State Special School,file_year,pct_met_exceeded
0,55.881991,3,Public,1056.0,ELA,2,Anderson County High School,10.0,9.504,6.336,251.328,0.000,11.616,3.168,1025.376,1.056,508.992,547.008,122.496,162.0,1,0,0,0,2019,40.544790
1,50.630516,4,Public,1078.0,ELA,2,Anderson County High School,10.0,14.014,10.780,257.642,3.234,11.858,6.468,1031.646,2.156,526.064,551.936,154.154,235.0,1,0,0,0,2021,30.170787
2,56.702652,5,Public,1095.0,ELA,2,Anderson County High School,10.0,0.000,0.000,251.850,0.000,0.000,0.000,1040.250,0.000,537.645,557.355,154.395,245.0,1,0,0,0,2022,36.082407
3,20.795153,22,Public,310.0,ELA,5,Andersonville Elementary,10.0,5.890,0.000,113.150,0.000,3.100,3.100,297.910,0.930,141.050,168.950,62.930,26.0,1,0,0,0,2019,39.500000
4,19.670934,23,Public,296.0,ELA,5,Andersonville Elementary,10.0,5.920,0.000,105.080,0.000,5.032,0.888,283.864,2.960,158.952,137.048,42.920,15.0,1,0,0,0,2021,42.300000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5575,-3.341631,27552,Public Charter,460.0,ELA,8140,Hillcrest High School,985.0,452.180,0.000,231.840,0.000,7.820,0.000,0.000,15.180,224.940,235.060,82.800,288.0,0,1,0,0,2021,7.607614
5576,2.922921,27553,Public Charter,451.0,ELA,8140,Hillcrest High School,985.0,428.450,0.000,234.971,0.000,0.000,0.000,0.000,27.962,222.794,228.206,86.141,245.0,0,1,0,0,2022,4.424020
5577,-0.160918,27563,Public Charter,156.0,ELA,8005,Bluff City High School,986.0,122.928,0.000,100.932,0.000,31.980,0.000,0.936,21.996,80.964,75.036,34.944,90.0,0,1,0,0,2019,9.327083
5578,-10.618931,27564,Public Charter,416.0,ELA,8005,Bluff City High School,986.0,331.968,2.912,215.904,0.000,81.120,0.000,0.000,64.064,193.024,222.976,69.056,324.0,0,1,0,0,2021,6.108696


In [35]:
with pd.ExcelWriter('../data_for_analysis/correlations.xlsx') as writer:  
    df_econ_v_percent.to_excel(writer, sheet_name='econ_v_percent')
    df_poc_v_percent.to_excel(writer, sheet_name='poc_v_percent')
    summary_math_w_school_type_df.to_excel(writer, sheet_name='math_summary_w_school_type')
    summary_math_no_school_type_df.to_excel(writer, sheet_name='math_summary_wo_school_type')
    predictions_math_no_school_type_df.to_excel(writer, sheet_name='math_results_wo_school_type')
    summary_ela_w_school_type_df.to_excel(writer, sheet_name='ela_summary_w_school_type')
    summary_ela_no_school_type_df.to_excel(writer, sheet_name='ela_summary_wo_school_type')
    predictions_ela_no_school_type_df.to_excel(writer, sheet_name='ela_results_wo_school_type')